In [25]:
from cvxopt import matrix
from collections import namedtuple
import math
Point = namedtuple("Point", ['x', 'y'])
Facility = namedtuple("Facility", ['index', 'setup_cost', 'capacity', 'location'])
Customer = namedtuple("Customer", ['index', 'demand', 'location'])
def length(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)

In [11]:
from cvxpy import *
Bool()

Bool(1, 1)

In [74]:
#variables
facilities = [Facility(0, 100, 100, Point(1065.0, 1065.0)),
             Facility(1, 100, 100, Point(1062.0, 1062.0)),
             Facility(2, 100, 500, Point(0.0, 0.0))]
customers = [Customer(0, 50, Point(1397.0, 1397.0)),
             Customer(1, 50, Point(1398.0, 1398.0)),
             Customer(2, 75, Point(1399.0, 1399.0)),
             Customer(3, 75, Point(586.0, 586.0))]


In [62]:
#variables
facilities = [Facility(0, 100, 100, Point(1065.0, 1065.0))]
customers = [Customer(0, 1, Point(1397.0, 1397.0)),
             Customer(1, 2, Point(1398.0, 1398.0)),
             Customer(2, 3, Point(1399.0, 1399.0)),
             Customer(3, 4, Point(586.0, 586.0))]


In [75]:
in_facility = [[Bool() for c in customers] for f in facilities]
# in_facility[facility][customer]

in_business = [Bool() for f in facilities]

    

In [76]:
M = 100000000
# under capacity
constraints = []
for i, f in enumerate(facilities):
    usage = None
    fac_bus = None
    for j, c in enumerate(customers):
        distance = length(f.location, c.location)
        if usage is None:
            usage = in_facility[i][j] * c.demand
        else:
            usage += in_facility[i][j] * c.demand
        if fac_bus is None:
            fac_bus = in_facility[i][j]
        else:
            fac_bus += in_facility[i][j]
            
    constraint = usage <= f.capacity
    constraints.append(constraint)
 
    # only in business can serve
    constraint = fac_bus <= in_business[i] * M
    constraints.append(constraint)
    

# only served by one
for j, c in enumerate(customers):
    served_by = None
    for i, f in enumerate(facilities):
        if served_by is None:
            served_by = in_facility[i][j]
        else:
            served_by += in_facility[i][j]

    constraint = served_by == 1
    constraints.append(constraint)
    
cost = None
for i, f in enumerate(facilities):
    for j, c in enumerate(customers):
        distance = length(f.location, c.location)
        if cost is None:
            cost = in_facility[i][j] * distance
        else:
            cost += in_facility[i][j] * distance

for i, f in enumerate(facilities):
    cost += in_business[i] * facilities[i].setup_cost

 

In [77]:
# Form objective.
obj = Minimize(cost)

# Form and solve problem.
prob = Problem(obj, constraints)
prob.solve()

2245.7711437985135

In [82]:
print(in_business[0].value)
print(in_business[1].value)
print(in_business[2].value)

2.27319777728e-08
1.48712325382e-08
1.48712201796e-08


In [87]:
print(in_facility[0][0].value)
print(in_facility[1][0].value)
print(in_facility[2][0].value)


0.99999961736
3.82587196099e-07
4.28271851683e-11


In [88]:
print(in_facility[0][1].value)
print(in_facility[1][1].value)
print(in_facility[2][1].value)

0.99999961736
3.82587181584e-07
4.28271850912e-11


In [89]:
print(in_facility[0][2].value)
print(in_facility[1][2].value)
print(in_facility[2][2].value)

4.53976536386e-07
0.999999545972
4.51174899149e-11


In [90]:
print(in_facility[0][3].value)
print(in_facility[1][3].value)
print(in_facility[2][3].value)

9.15471377077e-12
9.41963681067e-12
0.999999999982
